In [ ]:
import copy

import pandas as pd
import datasets

from datasets import Dataset, Image, ClassLabel, Sequence, Value, Features

from detection_dataset import Converter

In [ ]:
read_config = {
    'dataset_format': 'coco',
    'path': '/Users/jeromeblin/self/blinjrm/detection-dataset/data/raw',
    'splits': {
        'train': ('train.json', 'train'),
        'test': ('test.json', 'test'),
    },
}

converter = Converter()
converter.read(**read_config)

In [ ]:
c = copy.deepcopy(converter)
c.data.head(5)

In [ ]:
data = c.dataset.data_by_image
data.bbox = [[bbox.to_coco() for bbox in bboxes] for bboxes in data.bbox]
data.head(5)

In [ ]:
dataset = Dataset.from_pandas(data)

In [ ]:
dataset

In [ ]:
dataset[0]

In [ ]:
def export_to_hf_dataset(data: pd.DataFrame):
    hf_data = []
    
    for _, row in data.iterrows():
        objects = {}
        objects["id"] = row["category_id"]
        objects["category"] = row["category"]
        objects["bbox"] = row["bbox"]
        objects["area"] = row["area"]

        data = {}
        data["image_id"] = row["image_id"]
        data["image"] = row["image_path"]
        data["width"] = row["width"]
        data["height"] = row["height"]
        data["objects"] = objects

        hf_data.append(data)
    
    return hf_data

d = export_to_hf_dataset(data)

In [ ]:
cat = c.dataset.category_names

In [ ]:
x = pd.DataFrame.from_dict(d)
d_hf = dataset.from_pandas(x).cast_column("image", Image())
d_hf

In [ ]:
d_hf.features

In [ ]:
features = Features(
    {
        "image_id": Value(dtype='int64'),
        "image": Image(decode=True),
        "width": Value(dtype='int64'),
        "height": Value(dtype='int64'),
        "objects": Sequence(
            {
                "area": Value(dtype='int64'),
                "bbox": Sequence(feature=Value(dtype='float64'), length=-1),
                "category": ClassLabel(names=list(c.dataset.category_names)),
                "id": Value(dtype='int64'),
            }
        ),
    }
)

d_hf2 = dataset.from_pandas(df=x, features=features, split='train')

In [ ]:
d_hf2.features

In [ ]:
d_hf[0]

In [ ]:
d_hf.features['objects']

In [ ]:
d_hf.features

In [ ]:
d_hf.features['image']

In [ ]:
d_hf.shuffle(seed=42).select(range(1000))

In [ ]:
im = d_hf[0]['image']
im.filename

In [ ]:
d_hf.to_pandas()

In [ ]:
d = d_hf.to_pandas()
d.loc[0, 'image']

In [ ]:
pd.json_normalize(x['objects'])

In [ ]:
d_hf.push_to_hub('detection-dataset/fashionpedia_train')